In [1]:
import numpy as np
import matplotlib.pyplot as plt
import shapely.geometry as sha
from pylab import rcParams
%matplotlib inline

# change size of all figures in this notebook
rcParams['figure.figsize'] = 18, 8

def plotLactate(VO2max, VLamax, RunningEco, Ks, Kss, VolRel):
    
    # Generate oxygen steady-states
    VO2ss = np.arange(1,VO2max,0.01)
    
    # potentiate constants
    Ks=Ks**2
    Kss=Kss**3
    
    # calculate ADP corresponding to VO2ss (eq. 4b)
    ADP = np.sqrt((Ks * VO2ss) / (VO2max - VO2ss)) 
    
    # calculate steady state gross lactic acid (pyruvate) formation rate (eq. 3)
    dladtss = 60 * VLamax / (1 + ( Kss / ADP ** 3))

    # calculate lactate combustion (eq. 6)
    La_comb = (0.01576/VolRel) * VO2ss

    # calculate net lactate formation
    dladtnet = abs(dladtss - La_comb)
    
    # calculate the crossing point between gross lactate production and combustion
    line_1 = sha.LineString(np.column_stack((dladtss, VO2ss)))
    line_2 = sha.LineString(np.column_stack((La_comb, VO2ss)))
    intersection = line_1.intersection(line_2)
    
    # speed at anaerobic threshold
    sAT = intersection.y / RunningEco 

    # percentage of VO2max
    pcVO2maxAT = intersection.y / VO2max
    
    # print to console
    print(f'AT is at {round(sAT,2)} m/s, {round(pcVO2maxAT*100,1)}% of VO2max')
        
    # convert VO2ss into velocity (m/s)
    v = VO2ss / RunningEco
    
    # plot all lactate curves
    fig, ax = plt.subplots()
    plt.plot(v, dladtnet, color='green')
    plt.plot(v, dladtss, color='red')
    plt.plot(v, La_comb, color='blue')
    
    plt.legend(['Lack of Pyruvate + Lactate acummulation', 
                'Gross Lactate Formation', 
                'Potential Lactate Removal'])
    
    # set limits for interactive visualisation
    plt.ylim(0, 8)
    plt.xlim( 0, 6 )
    
    ## highlight the crossing point in red
    # plt.plot(round(intersection.y,1),round(intersection.x,1), 'ro')
    # plt.plot(intersection.y,intersection.x, 'ro')
    
    # vertical line at AT
    
    plt.axvline(x=sAT, linestyle='dotted', color='black')
    ax.set_xlabel('speed in [m/s]')
    ax.set_ylabel('mmol/l/min')
    plt.legend(['Lack of Pyruvate + Lactate acummulation', 
                'Gross Lactate Formation', 
                'Potential Lactate Removal','AT'])
    
import ipywidgets as widgets

widgets.interact(plotLactate, 
         VO2max = widgets.FloatSlider(value=55, min=40, max=80,step=0.5),
         VLamax = widgets.FloatSlider(value=0.5, min=0.2, max=0.8,step=0.01),
         RunningEco = widgets.FloatSlider(value=11.95, min=10, max=15,step=0.05),
         Ks = widgets.FloatSlider(value=0.25 , min=0.2, max=0.3,step=0.01),
         Kss = widgets.FloatSlider(value=1.2, min=1, max=1.3,step=0.01),
         VolRel = widgets.FloatSlider(value=.45, min=.42, max=.45,step=0.01))

interactive(children=(FloatSlider(value=55.0, description='VO2max', max=80.0, min=40.0, step=0.5), FloatSlider…

<function __main__.plotLactate(VO2max, VLamax, RunningEco, Ks, Kss, VolRel)>